# 🌍 GraphRAG Core: Climate Intelligence Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nunezmatias/grafoRag/blob/main/examples/Tutorial_GraphRAG.ipynb)

Welcome to the **GraphRAG Core** tutorial. This notebook demonstrates a next-generation retrieval system designed for scientific discovery. Unlike traditional search engines that return isolated documents, this system understands the *structure* of knowledge.

By combining vector search with a causal knowledge graph, we can answer complex questions about climate adaptation, identifying not just *what* is happening, but *why* it matters and what ripple effects it might trigger.

## 1. Setup & Installation
We will install the `graphrag_core` library directly from the repository. This package includes the retrieval engine and automatically handles the download of the Climate Knowledge Graph (~300MB).

In [ ]:
!pip install git+https://github.com/nunezmatias/grafoRag.git
!pip install -q -U google-genai

import os
from graphrag_core import GraphRAGEngine
print("✅ Libraries Installed & Loaded")

## 2. Initialize the Engine
Initializing the engine is simple. If the climate data is not found locally, it will be automatically downloaded from the cloud storage. This ensures you have the latest version of the knowledge graph.

In [ ]:
engine = GraphRAGEngine()
# Watch the output below for the download progress bar

## 3. Run a Deep Research Query
We will now perform a complex search. The engine allows you to tune the depth of the investigation:

- **`top_k`** controls breadth (how many distinct topics to start with).
- **`context_k`** controls depth (how many papers to read per topic).
- **`hops`** controls causal reasoning (how far to travel in the graph).

A configuration of `hops=2` allows us to see second-order effects, essential for understanding cascading risks.

In [ ]:
# Define your research question
query = "cascading risks of extreme heat and urban floods"

# Execute the Search
results = engine.search(
    query=query, 
    top_k=3,        # Breadth
    context_k=4,    # Depth
    hops=2          # Causality
)

print(f"--- Research Stats ---")
print(f"Primary Sources: {results['stats']['primary']}")
print(f"Context Expansion: {results['stats']['context']}")
print(f"Causal Links:      {results['stats']['graph']}")

## 4. Inspect the Intelligence
It is important to verify the quality of the retrieved data. Below, we print the top-ranked paper and a sample of the causal chains discovered by the graph traversal.

In [ ]:
# 1. Check the Top Paper
if results['papers']:
    p = results['papers'][0]
    print(f"📄 Top Paper: {p['title']}")
    print(f"   Snippet: {p['content'][:200]}...")

# 2. Check Discovered Causal Chains
if results['graph_links']:
    print("
🔗 Sample Causal Chains:")
    for link in results['graph_links'][:5]:
        print(f"   {link['node1']} --[{link['relation']}]--> {link['node2']}")

## 5. Construct the Expert Prompt
We use the engine's built-in expert template to package this structured data into a rigorous prompt for the LLM. This template forces the model to triangulate evidence and cite sources.

In [ ]:
# Using the default expert template designed for this Climate Graph
prompt = engine.format_prompt(results, query)

print("Here is your optimized prompt (COPY THIS):")
print("--------------------------------------------------")
print(prompt)
print("--------------------------------------------------")

## 6. Generate Answer with Gemini Flash ⚡
Finally, we send the generated prompt to Google's Gemini model. 

> **Tip:** If you want to modify the prompt (e.g., change the role or add specific constraints), simply edit the string in the `prompt` variable before running this cell.

**Prerequisite:** Add your API Key to Colab Secrets (Key icon on the left) with the name `GOOGLE_API_KEY`.

In [ ]:
from google import genai
from google.colab import userdata
from IPython.display import Markdown, display

# Configuración de la API Key
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print("✅ Gemini Client Configured")
except Exception as e:
    print("⚠️ Error: API Key not found. Please add 'GOOGLE_API_KEY' to Colab Secrets.")

# Generar contenido
print("⏳ Generating expert response with Gemini 2.0 Flash...")
try:
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt
    )
    
    # Mostrar la respuesta formateada
    display(Markdown("### 🤖 Response:"))
    display(Markdown(response.text))
except Exception as e:
    print(f"❌ Generation Error: {e}")